In [1]:
pwd

u'/home/houruijie/traffic/traffic/src'

In [13]:
#encoding:utf-8
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
plt.figure(figsize=(15,18))
import os
import sys
import re
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import xgboost as xgb
%matplotlib inline
predict_dir_path = '/home/houruijie/traffic/traffic/data/predict_data/'
train_dir_path = '/home/houruijie/traffic/traffic/data/train/'

In [3]:
def concat_goods_counts():
    #将读取的第一个CSV文件写入合并后的文件保存
    file_list = os.listdir(train_dir_path + '货量表/')
    df = pd.read_csv(train_dir_path+ '货量表/' + file_list[0]) 
    df.to_csv(train_dir_path + 'goods_counts.csv',index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv(train_dir_path + '货量表/' + file_list[i])
        df.to_csv(train_dir_path + 'goods_counts.csv',index=False, header=False, mode='a+')
concat_goods_counts()

In [4]:
def concat_call_logs():
    #将读取的第一个CSV文件写入合并后的文件保存
    file_list = os.listdir(train_dir_path + '司机电话日志/')
    df = pd.read_csv(train_dir_path+ '司机电话日志/' + file_list[0]) 
    df.to_csv(train_dir_path + 'call_logs.csv',index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv(train_dir_path + '司机电话日志/' + file_list[i])
        df.to_csv(train_dir_path + 'call_logs.csv',index=False, header=False, mode='a+')
driver_call_logs = pd.read_csv(train_dir_path + 'call_logs.csv' )

In [5]:
# 司机画像
driver_portrait = pd.read_csv(train_dir_path + '司机画像.csv')
# 货主画像
shipper_portrait = pd.read_csv(train_dir_path+'货主画像.csv')
# 城市公路距离
city_way_distance = pd.read_csv(train_dir_path + '城市公路距离记录.csv')
# 城市天气记录
city_weather_history = pd.read_csv(train_dir_path + '城市天气记录.csv', encoding='gbk')
city_weather_history.date = pd.to_datetime(city_weather_history['date'], format='%Y-%m-%d', errors='ignore')
# 货量表
goodscounts = pd.read_csv(train_dir_path + 'goods_counts.csv')
goodscounts.day = pd.to_datetime(goodscounts['day'], format='%Y%m%d', errors='ignore')
# 司机电话日志
driver_call_logs = pd.read_csv(train_dir_path + 'call_logs.csv' )
# 司机量表
driver_counts = pd.read_csv(train_dir_path + 'driver_counts.csv')
driver_counts.day = pd.to_datetime(driver_counts['day'], format='%Y%m%d', errors='ignore')


In [6]:
# merge weather data
goods_counts_add_weather = goodscounts.merge(city_weather_history, left_on=['city', 'day'], right_on=['code', 'date'],)
driver_counts_add_weather = driver_counts.merge(city_weather_history, left_on=['city', 'day'], right_on=['code', 'date'])
goods_counts_add_weather['is_train'] = 1
driver_counts_add_weather['is_train'] = 1

In [7]:
# 预测数据
# predict = pd.read_csv(predict_dir_path + 'predict.csv', header=None)
weather = pd.read_csv(predict_dir_path + '天气数据.csv', encoding='gbk')
weather.date = pd.to_datetime(weather['date'], format='%Y-%m-%d', errors='ignore')
# predict_sample = pd.read_csv(predict_dir_path + 'predict_sample.csv', header=None)
predict_data = pd.read_csv(predict_dir_path + 'sample.csv')
predict_data.day = pd.to_datetime(predict_data['day'], format='%Y%m%d', errors='ignore')
predict_data = predict_data.merge(weather, left_on=['city', 'day'], right_on=['code', 'date'])
predict_data['is_train'] = 0

In [8]:
goods_counts_features = ['city', 'day', 'weather', 'temperature','wind','is_train']
driver_counts_features = ['city', 'day', 'weather', 'temperature','wind','is_train']

In [9]:
def get_max_temperature(string):
    return int(string.split('/')[0])
def get_min_temperature(string):
    return int(string.split('/')[1])
def get_diff_temperature(string):
    return int(string.split('/')[0]) - int(string.split('/')[1])
# wind
def get_number_wind(string):
    pattern = re.compile(ur'\d')
    return max(map(int, list(pattern.findall(string))))
def feature_engineering(goods_counts_add_weather):
    # date
    goods_counts_add_weather['weekday'] = goods_counts_add_weather.day.dt.weekday
    goods_counts_add_weather['monthofday'] = goods_counts_add_weather.day.dt.day
    # temperature
    goods_counts_add_weather['max_temperature'] = goods_counts_add_weather.temperature.apply(get_max_temperature)
    goods_counts_add_weather['min_temperature'] = goods_counts_add_weather.temperature.apply(get_min_temperature)
    goods_counts_add_weather['diff_temperature'] = goods_counts_add_weather.temperature.apply(get_diff_temperature)
    # weather
    count_vect = CountVectorizer()
    weather = pd.DataFrame(count_vect.fit_transform(goods_counts_add_weather["weather"]).toarray(), columns=count_vect.get_feature_names())
    goods_counts_add_weather = pd.concat([goods_counts_add_weather, weather],axis=1)
    # wind
    goods_counts_add_weather['wind'] = goods_counts_add_weather.wind.apply((get_number_wind))
    # feature select
    no_use_featuer = ['day', 'name', 'code', 'date','weather', 'temperature']
    feature = [x for x in goods_counts_add_weather if x not in no_use_featuer]
    return goods_counts_add_weather[feature]

In [10]:
goods_counts_data = feature_engineering(pd.concat([goods_counts_add_weather[goods_counts_features], predict_data[goods_counts_features]], ignore_index=True))
goods_counts_train_data = goods_counts_data[goods_counts_data.is_train==1]
goods_counts_predict_data = goods_counts_data[goods_counts_data.is_train==0]

In [11]:
driver_counts_data = feature_engineering(pd.concat([driver_counts_add_weather[driver_counts_features], predict_data[driver_counts_features]], ignore_index=True))
driver_counts_train_data = driver_counts_data[driver_counts_data.is_train==1]
driver_counts_predict_data = driver_counts_data[driver_counts_data.is_train==0]

## goos_counts_prediction

In [12]:
no_use_feature = ['is_train']
features = [x for x in goods_counts_train_data.columns if x not in no_use_feature]
train_data = goods_counts_train_data[features]
test = goods_counts_predict_data[features]
label = goods_counts_add_weather['count']

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5
    

lgb_params1 = {}
lgb_params1['application'] = 'regression'
lgb_params1['boosting'] = 'gbdt'
lgb_params1['learning_rate'] = 0.015
lgb_params1['num_leaves'] = 32
lgb_params1['min_sum_hessian_in_leaf'] = 2e-2
lgb_params1['min_gain_to_split'] = 0
lgb_params1['bagging_fraction'] = 0.9
lgb_params1['feature_fraction'] = 0.9
lgb_params1['num_threads'] = 4
lgb_params1['metric'] = 'rmse'

lgb_params2 = {}
lgb_params2['application'] = 'regression'
lgb_params2['boosting'] = 'gbdt'
lgb_params2['learning_rate'] = 0.02
lgb_params2['lambda_l1'] = 0.5
lgb_params2['num_leaves'] = 32
lgb_params2['min_gain_to_split'] = 0
lgb_params2['bagging_fraction'] = 0.8
lgb_params2['feature_fraction'] = 0.8
lgb_params2['num_threads'] = 4
lgb_params2['metric'] = 'rmse'

lgb_params3 = {}
lgb_params3['application'] = 'regression'
lgb_params3['boosting'] = 'gbdt'
lgb_params3['learning_rate'] = 0.022
lgb_params3['num_leaves'] = 32
lgb_params2['lambda_l2'] = 0.3
lgb_params3['bagging_freq'] = 8
lgb_params3['min_gain_to_split'] = 0
lgb_params3['bagging_fraction'] = 0.8
lgb_params3['feature_fraction'] = 0.8
lgb_params3['num_threads'] = 4
lgb_params3['metric'] = 'rmse'

def do_train(X_train, X_valid, y_train, y_valid, lgb_params, rounds):
    X_t = X_train.values
    y_t = y_train.values
    d_train = lgb.Dataset(X_t, y_t)
    X_v = X_valid.values
    y_v = y_valid.values
    d_valid = lgb.Dataset(X_v, y_v)
    watchlist = [d_train, d_valid]
    lgb_model = lgb.train(lgb_params, train_set=d_train, num_boost_round=rounds, 
                          valid_sets=watchlist, verbose_eval=1000, early_stopping_rounds = 300)
    test_pred = lgb_model.predict(X_v)
    rmsle = RMSLE(y_v, test_pred)
#     print(X_t)
#     print(lgb_model.feature_importance())
    return rmsle, lgb_model

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=74)
rmsle, lgb_model1 = do_train(X_train, X_valid, y_train, y_valid, lgb_params1, 12000)
test_pred1 = lgb_model1.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)
    
X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=2121)
rmsle, lgb_model2 = do_train(X_train, X_valid, y_train, y_valid, lgb_params2, 10000)
test_pred2 = lgb_model2.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)   

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=4)
rmsle, lgb_model3 = do_train(X_train, X_valid, y_train, y_valid, lgb_params3, 8000)
test_pred3 = lgb_model3.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)   

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=19)
rmsle, lgb_model4 = do_train(X_train, X_valid, y_train, y_valid, lgb_params3, 8000)
test_pred4 = lgb_model4.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)  

#test_pred = (test_pred3 + test_pred4) / 2
test_pred = (test_pred1 + test_pred2 + test_pred3 + test_pred4) / 4
# result.to_csv('LGB_sub.csv', index=False)
print('Done')

Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 129.833	valid_1's rmse: 278.427
Early stopping, best iteration is:
[999]	training's rmse: 129.914	valid_1's rmse: 278.374
Test RMSLE: 278.374
Training until validation scores don't improve for 300 rounds.
Early stopping, best iteration is:
[671]	training's rmse: 143.493	valid_1's rmse: 293.003
Test RMSLE: 293.003
Training until validation scores don't improve for 300 rounds.
Early stopping, best iteration is:
[495]	training's rmse: 185.48	valid_1's rmse: 288.097
Test RMSLE: 288.097
Training until validation scores don't improve for 300 rounds.
Early stopping, best iteration is:
[552]	training's rmse: 171.693	valid_1's rmse: 301.741
Test RMSLE: 301.741
Done


In [29]:
submit = pd.read_csv(predict_dir_path + 'sample.csv')
submit['cargo_count'] = test_pred.astype(int)


## driver_call_counts

In [31]:
no_use_feature = ['is_train']
features = [x for x in driver_counts_train_data.columns if x not in no_use_feature]
train_data = driver_counts_train_data[features]
test = driver_counts_predict_data[features]
label = driver_counts_add_weather['driver_call_count']

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

def RMSLE(y, pred):
    return mean_squared_error(y, pred)**0.5

lgb_params1 = {}
lgb_params1['application'] = 'regression'
lgb_params1['boosting'] = 'gbdt'
lgb_params1['learning_rate'] = 0.015
lgb_params1['num_leaves'] = 32
lgb_params1['min_sum_hessian_in_leaf'] = 2e-2
lgb_params1['min_gain_to_split'] = 0
lgb_params1['bagging_fraction'] = 0.9
lgb_params1['feature_fraction'] = 0.9
lgb_params1['num_threads'] = 4
lgb_params1['metric'] = 'rmse'

lgb_params2 = {}
lgb_params2['application'] = 'regression'
lgb_params2['boosting'] = 'gbdt'
lgb_params2['learning_rate'] = 0.02
lgb_params2['lambda_l1'] = 0.5
lgb_params2['num_leaves'] = 32
lgb_params2['min_gain_to_split'] = 0
lgb_params2['bagging_fraction'] = 0.8
lgb_params2['feature_fraction'] = 0.8
lgb_params2['num_threads'] = 4
lgb_params2['metric'] = 'rmse'

lgb_params3 = {}
lgb_params3['application'] = 'regression'
lgb_params3['boosting'] = 'gbdt'
lgb_params3['learning_rate'] = 0.022
lgb_params3['num_leaves'] = 32
lgb_params2['lambda_l2'] = 0.3
lgb_params3['bagging_freq'] = 8
lgb_params3['min_gain_to_split'] = 0
lgb_params3['bagging_fraction'] = 0.8
lgb_params3['feature_fraction'] = 0.8
lgb_params3['num_threads'] = 4
lgb_params3['metric'] = 'rmse'

def do_train(X_train, X_valid, y_train, y_valid, lgb_params, rounds):
    X_t = X_train.values
    y_t = y_train.values
    d_train = lgb.Dataset(X_t, y_t)
    X_v = X_valid.values
    y_v = y_valid.values
    d_valid = lgb.Dataset(X_v, y_v)
    watchlist = [d_train, d_valid]
    lgb_model = lgb.train(lgb_params, train_set=d_train, num_boost_round=rounds, 
                          valid_sets=watchlist, verbose_eval=1000, early_stopping_rounds = 300)
    test_pred = lgb_model.predict(X_v)
    rmsle = RMSLE(y_v, test_pred)
#     print(X_t)
#     print(lgb_model.feature_importance())
    return rmsle, lgb_model

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=74)
rmsle, lgb_model1 = do_train(X_train, X_valid, y_train, y_valid, lgb_params1, 12000)
test_pred1 = lgb_model1.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)
    
X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=2121)
rmsle, lgb_model2 = do_train(X_train, X_valid, y_train, y_valid, lgb_params2, 10000)
test_pred2 = lgb_model2.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)   

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=4)
rmsle, lgb_model3 = do_train(X_train, X_valid, y_train, y_valid, lgb_params3, 8000)
test_pred3 = lgb_model3.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)   

X_train, X_valid, y_train, y_valid = train_test_split(train_data, label, test_size=0.3, random_state=19)
rmsle, lgb_model4 = do_train(X_train, X_valid, y_train, y_valid, lgb_params3, 8000)
test_pred4 = lgb_model4.predict(test.values)
print('Test RMSLE: %.3f' % rmsle)  

#test_pred = (test_pred3 + test_pred4) / 2
test_pred = (test_pred1 + test_pred2 + test_pred3 + test_pred4) / 4
# result.to_csv('LGB_sub.csv', index=False)
print('Done')

Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 400.447	valid_1's rmse: 820.132
Early stopping, best iteration is:
[868]	training's rmse: 432.446	valid_1's rmse: 818.738
Test RMSLE: 818.738
Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 337.394	valid_1's rmse: 940.287
Early stopping, best iteration is:
[1072]	training's rmse: 322.271	valid_1's rmse: 939.306
Test RMSLE: 939.306
Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 389.496	valid_1's rmse: 894.816
Early stopping, best iteration is:
[782]	training's rmse: 445.876	valid_1's rmse: 891.434
Test RMSLE: 891.434
Training until validation scores don't improve for 300 rounds.
[1000]	training's rmse: 353.635	valid_1's rmse: 1012.68
Early stopping, best iteration is:
[1146]	training's rmse: 322.632	valid_1's rmse: 1006.96
Test RMSLE: 1006.956
Done


In [34]:
submit['driver_call_count'] = test_pred.astype(int)

In [36]:
submit.to_csv('submit07101052.csv', header=False)